In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
train=pd.read_csv('/home/zebraking/Downloads/BEAUTIFULLSOUP/train.csv')

In [3]:
train.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [4]:
test=pd.read_csv('/home/zebraking/Downloads/BEAUTIFULLSOUP/test.csv')
test.head(2)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S


In [5]:
combine=[train,test]



In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [7]:
# to check null values
train.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [8]:
train[['Pclass','Survived']].groupby(['Pclass'],as_index=False).mean().sort_values(by='Survived')
                                     

,Pclass,Survived
2,3,0.242363
1,2,0.472826
0,1,0.629630


In [9]:
train[['Sex','Survived']].groupby(["Sex"],as_index=False).mean().sort_values(by="Survived")

,Sex,Survived
1,male,0.188908
0,female,0.742038


In [10]:
train[['Embarked','Survived']].groupby(['Embarked'],as_index=False).mean().sort_values(by='Survived')

,Embarked,Survived
2,S,0.336957
1,Q,0.389610
0,C,0.553571


In [11]:
train['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [12]:
#train[['Embarked','Fare']].groupby(['Embarked'],as_index=False).mean().sort_values(by='Fare')

In [13]:
#droping the features
train=train.drop(['Ticket','Cabin'],axis=1)
test=test.drop(['Ticket','Cabin'],axis=1)

In [14]:
combine=[train,test]

In [15]:
for dataset in combine:
    dataset['Title']=dataset.Name.str.extract('([A-Za-z]+)\.',expand=False)

In [16]:
pd.crosstab(train['Title'],train['Sex'])

Sex,female,male
Title,,
Capt,0,1
Col,0,2
Countess,1,0
Don,0,1
Dr,1,6
Jonkheer,0,1
Lady,1,0
Major,0,2
Master,0,40


In [17]:
for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
    'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    
    dataset['Title']=dataset['Title'].replace('Mlle','Miss')
        
    dataset['Title']=dataset['Title'].replace('Ms','Miss')
    dataset['Title']=dataset['Title'].replace('Mme','Mrs')


In [18]:
train[['Title','Survived']].groupby(['Title'],as_index=False).mean()

,Title,Survived
0,Master,0.575000
1,Miss,0.702703
2,Mr,0.156673
3,Mrs,0.793651
4,Rare,0.347826


In [19]:
# we cant convert the categorcal to ordinal
title_mapping={'Mr':1,'Miss':2,'Mrs':3,'Master':4,'Rare':5}
for dataset in combine:
    dataset['Title']=dataset['Title'].map(title_mapping)
    dataset['Title']=dataset['Title'].fillna(0)
train.head()
    

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C,3
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S,2
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S,3
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S,1


In [20]:
# now we can drop the name and passengerId from training and testing dataset
train=train.drop(['Name','PassengerId'],axis=1)
test=test.drop(['Name'],axis=1)
combine=[train,test]

In [21]:
#Let us start by converting Sex feature to a new feature called Gender where female=1 and male=0.
for dataset in combine:
    dataset['Sex']=dataset['Sex'].map({'female':1,'male':0}).astype(int)

In [22]:
guess_ages = np.zeros((2,3))
guess_ages

array([[0., 0., 0.],
       [0., 0., 0.]])

In [23]:
import random as rnd

In [24]:
for i in range(0, 2):
        for j in range(0, 3):
            guess_df = dataset[(dataset['Sex'] == i) & \
                                  (dataset['Pclass'] == j+1)]['Age'].dropna()

            age_mean = guess_df.mean()
            age_std = guess_df.std()
            age_guess = rnd.uniform(age_mean - age_std, age_mean + age_std)

            age_guess = guess_df.median()

            # Convert random age float to nearest .5 age
            guess_ages[i,j] = int( age_guess/0.5 + 0.5 ) * 0.5
            
        for i in range(0, 2):
            for j in range(0, 3):
                dataset.loc[ (dataset.Age.isnull()) & (dataset.Sex == i) & (dataset.Pclass == j+1),\
                    'Age'] = guess_ages[i,j]

        dataset['Age'] = dataset['Age'].astype(int)

In [25]:
#Let us create Age bands and determine correlations with Survived.
train['Age'].fillna(train['Age'].dropna().median(), inplace=True)


In [26]:
train['Age']=train['Age'].astype(int)

In [27]:
train['AgeBand']=pd.cut(train['Age'],5)

In [28]:
train[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True)

,AgeBand,Survived
0,"(-0.08, 16.0]",0.550000
1,"(16.0, 32.0]",0.344762
2,"(32.0, 48.0]",0.403226
3,"(48.0, 64.0]",0.434783
4,"(64.0, 80.0]",0.090909


In [29]:
#Let us replace Age with ordinals based on these bands.
for dataset in combine:
    dataset.loc[dataset['Age'] <= 16, 'Age']=0
    dataset.loc[(dataset['Age'] >  16) & (dataset['Age'] <= 32), 'Age']=1
    dataset.loc[(dataset['Age'] >  32) & (dataset['Age'] <= 48), 'Age']=2
    dataset.loc[(dataset['Age'] >  48) & (dataset['Age'] <= 64), 'Age']=3
    dataset.loc[dataset['Age'] >  64 ,'Age']
train.head()

    

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,AgeBand
0,0,3,0,1,1,0,7.2500,S,1,"(16.0, 32.0]"
1,1,1,1,2,1,0,71.2833,C,3,"(32.0, 48.0]"
2,1,3,1,1,0,0,7.9250,S,2,"(16.0, 32.0]"
3,1,1,1,2,1,0,53.1000,S,3,"(32.0, 48.0]"
4,0,3,0,2,0,0,8.0500,S,1,"(32.0, 48.0]"


In [30]:
# we can remove the age band
train=train.drop(['AgeBand'],axis=1)
combine=[train,test]

In [31]:
#We can create a new feature for FamilySize which combines Parch and SibSp. This
#will enable us to drop Parch and SibSp from our datasets.
for dataset in combine:
    dataset['FamilySize']=dataset['SibSp'] + dataset['Parch'] +1

In [32]:
train[['FamilySize', 'Survived']].groupby(['FamilySize'], as_index=False).mean().sort_values(by='Survived', ascending=False)


,FamilySize,Survived
3,4,0.724138
2,3,0.578431
1,2,0.552795
6,7,0.333333
0,1,0.303538
4,5,0.200000
5,6,0.136364
7,8,0.000000
8,11,0.000000


In [33]:
#We can create another feature called IsAlone.
for dataset in combine:
    dataset['isAlone']=0
    dataset.loc[dataset['FamilySize']==1,'isAlone']=1
train[['isAlone','Survived']].groupby(['isAlone'],as_index=False).mean()


,isAlone,Survived
0,0,0.505650
1,1,0.303538


In [34]:
#Let us drop Parch, SibSp, and FamilySize features in favor of IsAlone
train=train.drop(['Parch','SibSp','FamilySize'],axis=1)
#Let us drop Parch, SibSp, and FamilySize features in favor of IsAlone
test=test.drop(['Parch','SibSp','FamilySize'],axis=1)
combine=[train,test]
train.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,Title,isAlone
0,0,3,0,1,7.2500,S,1,0
1,1,1,1,2,71.2833,C,3,0
2,1,3,1,1,7.9250,S,2,1
3,1,1,1,2,53.1000,S,3,0
4,0,3,0,2,8.0500,S,1,1


In [35]:
#We can also create an artificial feature combining Pclass and Age.
for dataset in combine:
    dataset['Age*Class']=train.Age * train.Pclass
train.loc[:,['Age*Class','Age','Pclass']].head(5)

,Age*Class,Age,Pclass
0,3,1,3
1,2,2,1
2,3,1,3
3,2,2,1
4,6,2,3


In [36]:
train['Embarked']=train['Embarked'].fillna('S')

In [37]:
train['Embarked'].value_counts()

S    646
C    168
Q     77
Name: Embarked, dtype: int64

In [38]:
#Converting categorical feature to numeric
#We can now convert the EmbarkedFill feature by creating a new numeric Port feature.
for dataset in combine:
    dataset['Embarked']=dataset['Embarked'].map({'S':0,'C':1,'Q':2}).astype(int)

In [39]:
train.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,Title,isAlone,Age*Class
0,0,3,0,1,7.2500,0,1,0,3
1,1,1,1,2,71.2833,1,3,0,2
2,1,3,1,1,7.9250,0,2,1,3
3,1,1,1,2,53.1000,0,3,0,2
4,0,3,0,2,8.0500,0,1,1,6


In [40]:
test['Fare'].fillna(test['Fare'].dropna().median(), inplace=True)
test.head()

,PassengerId,Pclass,Sex,Age,Fare,Embarked,Title,isAlone,Age*Class
0,892,3,0,2,7.8292,2,1,1,3
1,893,3,1,2,7.0000,0,3,0,2
2,894,2,0,3,9.6875,2,1,1,3
3,895,3,0,1,8.6625,0,1,1,2
4,896,3,1,1,12.2875,0,3,0,6


In [41]:
train['FareBand']=pd.qcut(train['Fare'],4)
train[['FareBand', 'Survived']].groupby(['FareBand'], as_index=False).mean().sort_values(by='FareBand', ascending=True)

,FareBand,Survived
0,"(-0.001, 7.91]",0.197309
1,"(7.91, 14.454]",0.303571
2,"(14.454, 31.0]",0.454955
3,"(31.0, 512.329]",0.581081


In [42]:
for dataset in combine:
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)

train = train.drop(['FareBand'], axis=1)
combine = [train, test]
    
train.head(10)

,Survived,Pclass,Sex,Age,Fare,Embarked,Title,isAlone,Age*Class
0,0,3,0,1,0,0,1,0,3
1,1,1,1,2,3,1,3,0,2
2,1,3,1,1,1,0,2,1,3
3,1,1,1,2,3,0,3,0,2
4,0,3,0,2,1,0,1,1,6
5,0,3,0,1,1,2,1,1,3
6,0,1,0,3,3,0,1,1,3
7,0,3,0,0,2,0,4,0,0
8,1,3,1,1,1,0,3,0,3
9,1,2,1,0,2,1,3,0,0


In [43]:
#train['Age'].fillna(train['Age'].dropna().median(), inplace=True)
#train.head()

In [44]:
train.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,Title,isAlone,Age*Class
0,0,3,0,1,0,0,1,0,3
1,1,1,1,2,3,1,3,0,2
2,1,3,1,1,1,0,2,1,3
3,1,1,1,2,3,0,3,0,2
4,0,3,0,2,1,0,1,1,6


,PassengerId,Pclass,Sex,Age,Fare,Embarked,Title,isAlone,Age*Class
0,892,3,0,2,0,2,1,1,3
1,893,3,1,2,0,0,3,0,2
2,894,2,0,3,1,2,1,1,3
3,895,3,0,1,1,0,1,1,2
4,896,3,1,1,1,0,3,0,6


In [46]:
x_train=train.drop('Survived',axis=1)

In [47]:
y_train=train['Survived']

In [50]:
x_test=test.drop('PassengerId',axis=1).copy()

In [51]:
x_train.shape,y_train.shape,x_test.shape

((891, 8), (891,), (418, 8))

In [54]:
#losgistic regression
from sklearn.linear_model import LogisticRegression

In [61]:
logreg=LogisticRegression()
logreg.fit(x_train,y_train)
y_pred=logreg.predict(x_test)
acc_log=round(logreg.score(x_train,y_train)*100,2)
acc_log

80.36

In [68]:
coeff_df=pd.DataFrame(train.columns.delete(0))
coeff_df.columns=['Feature']
coeff_df['Correlation']=pd.Series(logreg.coef_[0])
coeff_df.sort_values(by='Correlation',ascending=False)


,Feature,Correlation
1,Sex,2.202342
5,Title,0.397801
2,Age,0.288328
4,Embarked,0.261296
6,isAlone,0.124535
3,Fare,-0.087262
7,Age*Class,-0.311385
0,Pclass,-0.748108


In [71]:
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [74]:
# support vector machine
svc=SVC()
svc.fit(x_train,y_train)
y_pred=svc.predict(x_test)
acc_svc=round(svc.score(x_train,y_train)*100,2)
acc_svc

78.34

In [75]:
# k neariest neighbours
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(x_train,y_train)
y_pred=knn.predict(x_test)
acc_knn=round(knn.score(x_train,y_train) * 100,2)
acc_knn
                           

85.3

In [76]:
# naive bayes
gaussian = GaussianNB()
gaussian.fit(x_train,y_train)
y_pred=gaussian.predict(x_test)
acc_gaussian=round(gaussian.score(x_train,y_train)* 100 ,2)
acc_gaussian

71.83

In [77]:
# Perceptron

perceptron = Perceptron()
perceptron.fit(x_train,y_train)
y_pred=perceptron.predict(x_test)
acc_perceptron=round(perceptron.score(x_train,y_train)*100,2)
acc_perceptron

73.29

In [78]:
# Linear SVC

linear_svc = LinearSVC()
linear_svc.fit(x_train, y_train)
Y_pred = linear_svc.predict(x_test)
acc_linear_svc = round(linear_svc.score(x_train, y_train) * 100, 2)
acc_linear_svc

/home/zebraking/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


79.01

In [79]:
# Stochastic Gradient Descent

sgd = SGDClassifier()
sgd.fit(x_train, y_train)
Y_pred = sgd.predict(x_test)
acc_sgd = round(sgd.score(x_train, y_train) * 100, 2)
acc_sgd

75.08

In [80]:
# Decision Tree

decision_tree = DecisionTreeClassifier()
decision_tree.fit(x_train, y_train)
Y_pred = decision_tree.predict(x_test)
acc_decision_tree = round(decision_tree.score(x_train, y_train) * 100, 2)
acc_decision_tree


86.98

In [82]:
# Random Forest

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(x_train, y_train)
Y_pred = random_forest.predict(x_test)
random_forest.score(x_train, y_train)
acc_random_forest = round(random_forest.score(x_train, y_train) * 100, 2)
acc_random_forest

86.98

In [86]:
models=pd.DataFrame({
    'Model':['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 'Perceptron', 
              'Stochastic Gradient Decent', 'Linear SVC', 
              'Decision Tree'],
    'scores':[acc_svc, acc_knn, acc_log, 
              acc_random_forest, acc_gaussian, acc_perceptron, 
              acc_sgd, acc_linear_svc, acc_decision_tree]
})
models.sort_values(by='scores',ascending=False)

,Model,scores
3,Random Forest,86.98
8,Decision Tree,86.98
1,KNN,85.30
2,Logistic Regression,80.36
7,Linear SVC,79.01
0,Support Vector Machines,78.34
6,Stochastic Gradient Decent,75.08
5,Perceptron,73.29
4,Naive Bayes,71.83


In [87]:
submission = pd.DataFrame({
        "PassengerId": test["PassengerId"],
        "Survived": y_pred
    })

In [88]:
submission

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,0
415,1307,0
416,1308,0
